<a href="https://colab.research.google.com/github/Taiga10969/Learn-the-basics/blob/main/Hugging_Face-transformers/Tokenizer_bs/Huggin_Face_Tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hugging Face Tokenizers を使ってみた
Transformer のモデルはトークンの数値表現に基づいて学習される為，トークン化のステップを正しく行うことは NLP プロジェクト全体にとってかなり重要なことである．<br>
Hugging Face Tokenizers は多数のトークン化戦略を提供しており，入力の正規化やモデルの出力を必要な形式に変換するなどの事前・事後処理のステップも全て行うことができる．<br>
本 notebook では，この Hugging Face Tokeninzer の使い方について記録しておく．

## 必要ライブラリのインポート，トークナイザーのロード
Hugging Face Transformers は便利な AutoTokenizer クラスを提供している．<br>
このクラスを用いて学習済みモデルに関連づけられたトークナイザーを素早くロードすることができる．<br>
ロードは，Hub 上のモデルの ID またはローカルファイルのパスを指定して `from_pretrained()` メソッドを呼び出すことで可能である．<br>
※ 本 notebook では，DistilBERTのトークナイザーをロードし使用する．



In [1]:
! pip install -q transformers
! pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 10.2 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer

model_ckpt = 'distilbert-base-uncased'
#model_ckpt = 't5-small'
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

※ AutoTokenizer クラスを使用せず，特定のクラスをロードしたい場合は以下のようにすることで可能．<br>
`from transformers import DistilBertTokenizer`<br>
`distilbert_tokenizer = DistilBertTokenizer.from_pretrained(model_ckpt)`

## 特殊トークンの表示
Tokenizer に含まれる特殊トークンの一覧を表示

In [3]:
# 特殊トークンの一覧を取得
special_tokens = tokenizer.special_tokens_map
print(special_tokens)

{'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}


## 文字列のトークン化（ id 化）
実際に，定義した `tokenizer` を用いて文字列を id に変換するプロセスを以下に示す．<br>
以下に示すように，入力した文字列は，単語毎に変換され `input_ids` フィールドで一意な整数にマップされていることが確認できる．<br>
また，`attention_mask` も提供されており，Transformer などのモデル内部において，パディングトークン領域を無視するためにモデル内で用いられる．

In [4]:
text = 'This notebook describes Hugging Face Tokenizer'

encoded_text = tokenizer(text)
print(encoded_text)

{'input_ids': [101, 2023, 14960, 5577, 17662, 2227, 19204, 17629, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}


## id から単語・文字列への再変換
id 化された文字列を，人間が読むことができる単語に戻し，入力したテキストへ再変換を行う．<br>
単語 id から文字の単語への変換は，以下に示すようにトークナイザーの `convert_ids_to_tokens()` メソッドを用いることで可能である．

In [5]:
# id → token
tokens = tokenizer.convert_ids_to_tokens(encoded_text.input_ids)
print(tokens)

['[CLS]', 'this', 'notebook', 'describes', 'hugging', 'face', 'token', '##izer', '[SEP]']


以上の出力から，今回の文字列中に含まれる "Tokenizer" という単語は，"token" と "izer" という二つのサブワードに分割されトークン化が行われていることが確認できる．サブワードに分割されトークン化が行われている場合は，## というプレフィックスが付いているおり，これは，直前の文字列が空白ではないことを意味している．

In [6]:
# token → string
string = tokenizer.convert_tokens_to_string(tokens)
print(string)

[CLS] this notebook describes hugging face tokenizer [SEP]


ここで，再変換されたテキストは，元のテキストと比較して，文字列の最初と最後に特殊トークン [CLS] と [SEP] が追加されていることが確認できる．これらの，特殊トークンは，モデルによって異なるが，その役割は系列の開始と終了を示すことである．また，大文字は全て小文字に正規化されていることも確認できる．

## AutoTokenizer の情報提供属性
Autotokenizer クラスはトークナイザーに関する情報を提供する属性をいくつか以下に示す．

In [7]:
# vacab_size
tokenizer.vocab_size

30522

In [8]:
# 対応するモデルの最大コンテキストサイズ
tokenizer.model_max_length

512

In [9]:
# モデルがフォワードパスで期待するフィールドの名前
tokenizer.model_input_names

['input_ids', 'attention_mask']

## データセット全体のトークン化
コーパス全体をトークン化するプロセスを以下に示す．


### データセットの用意
今回は，実際に emotion データセットを用いて，データセット全体にトークン化を適応するプロセスを以下に示す．<br>
そのために，始めにデータセットを用意する．<br>

※ Hugging Face Hub にないデータセットを用いる場合は，書籍「機械学習エンジニアのためのTransformers」のp.27を参考．

In [10]:
from datasets import list_datasets, load_dataset

all_datasets = list_datasets()
print(f"There ate {len(all_datasets)} datasets currently available on the Hub")
print(f"The first 10 are : {all_datasets[:10]}")

<ipython-input-10-b1242aea85d9>:3: FutureWarning: list_datasets is deprecated and will be removed in the next major version of datasets. Use 'huggingface_hub.list_datasets' instead.
  all_datasets = list_datasets()


There ate 46968 datasets currently available on the Hub
The first 10 are : ['acronym_identification', 'ade_corpus_v2', 'adversarial_qa', 'aeslc', 'afrikaans_ner_corpus', 'ag_news', 'ai2_arc', 'air_dialogue', 'ajgt_twitter_ar', 'allegro_reviews']


In [11]:
# emotion データセットをロード
emotions = load_dataset("emotion")
emotions

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset emotion downloaded and prepared to /root/.cache/huggingface/datasets/emotion/split/1.0.0/cca5efe2dfeb58c1d098e0f9eeb200e9927d889b5a03c67097275dfb5fe463bd. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [12]:
# データセットの作成
train_ds = emotions['train']
train_ds

Dataset({
    features: ['text', 'label'],
    num_rows: 16000
})

In [13]:
# データセットの初めの5つの要素を確認
train_ds[:5]

{'text': ['i didnt feel humiliated',
  'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake',
  'im grabbing a minute to post i feel greedy wrong',
  'i am ever feeling nostalgic about the fireplace i will know that it is still on the property',
  'i am feeling grouchy'],
 'label': [0, 0, 3, 2, 3]}

In [14]:
# ☆データセットオブジェクトの内部でどのようなデータ型が使われているのかの確認
print(train_ds.features)

{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'], id=None)}


### 複数の事例をトークン化するための関数を定義
事例のバッチに対してトークナイザーを適用する関数を作成する．<br>
`padding = True` : バッチ内で最も長い事例のサイズまでゼロで埋める<br>
`truncation = True` : モデルの最大コンテキストサイズまでじれを切り詰める

In [15]:
def tokenize(batch):
    return tokenizer(batch['text'], padding = True, truncation=True, max_length=128)

### 作成した関数を用いてデータセットをトークン化


In [16]:
# 試しに初めの2要素に対してトークン化した結果
print(tokenize(emotions['train'][:2]))

{'input_ids': [[101, 1045, 2134, 2102, 2514, 26608, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 1045, 2064, 2175, 2013, 3110, 2061, 20625, 2000, 2061, 9636, 17772, 2074, 2013, 2108, 2105, 2619, 2040, 14977, 1998, 2003, 8300, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


In [17]:
# map()関数を用いてコーパスの全てに対して個別に適用させる
## batched=True : 一括してエンコード可能
## batch_size = None : tokenize()関数はデータセット全体を1つのバッチとして適用

emotions_encoded = emotions.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [18]:
print(emotions_encoded['train'].column_names)

['text', 'label', 'input_ids', 'attention_mask']


## 参考

Tunstall & Werra & Wolf, 機械学習エンジニアのためのTransformers, オライリー・ジャパン, 2022